# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [1]:
# import required libraries
import numpy as np
import pandas as pd
import zipfile

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [2]:
with zipfile.ZipFile('Orders.zip','r') as zip_ref:
    zip_ref.extractall()

orders = pd.read_csv('Orders.csv')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [3]:
#Sub Problem 1:

#grouping the customers by their customer ID and calculating their total amount_spent
customers_spent = orders.groupby(['CustomerID']).agg({'amount_spent':'sum'}).reset_index().sort_values('amount_spent',ascending=False)
customers_spent

,CustomerID,amount_spent
1690,14646,280206.02
4202,18102,259657.30
3729,17450,194550.79
3009,16446,168472.50
1880,14911,143825.06
...,...,...
4099,17956,12.75
3015,16454,6.90
1794,14792,6.20
3218,16738,3.75


In [4]:
#Sub Problem 2:

#identifing the percentile of the customers based on their total amount_spent
customers_spent['amount_spent'].quantile(q=(0.95,0.75))

0.95    5840.182
0.75    1661.640
Name: amount_spent, dtype: float64

In [5]:
#identifing the customers of the specific groups
customers_vip = customers_spent[customers_spent['amount_spent']>=5840.182]
customers_preferred = customers_spent[(customers_spent['amount_spent']>=1661.640) & (customers_spent['amount_spent']<5840.182)]
display(customers_vip.head())
display(customers_preferred.head())

,CustomerID,amount_spent
1690,14646,280206.02
4202,18102,259657.30
3729,17450,194550.79
3009,16446,168472.50
1880,14911,143825.06


,CustomerID,amount_spent
537,13050,5836.86
304,12720,5781.73
2105,15218,5756.89
3904,17686,5739.46
631,13178,5725.47


In [6]:
#Sub Problem 3:

#assigning labels to customer
customers_spent['customer_label'] = pd.qcut(customers_spent['amount_spent'],[0.0,.75,.95,1.],labels=['Standard','Preferred','VIP'])
customers_spent

,CustomerID,amount_spent,customer_label
1690,14646,280206.02,VIP
4202,18102,259657.30,VIP
3729,17450,194550.79,VIP
3009,16446,168472.50,VIP
1880,14911,143825.06,VIP
...,...,...,...
4099,17956,12.75,Standard
3015,16454,6.90,Standard
1794,14792,6.20,Standard
3218,16738,3.75,Standard


In [7]:
#adding the labels to the original dataset
orders = orders.merge(customers_spent,
             how='left',
             left_on='CustomerID',
             right_on='CustomerID')
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent_x,amount_spent_y,customer_label
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,5391.21,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,5391.21,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,5391.21,Preferred


In [8]:
#cleaning the dataset
orders = orders.drop(labels=['amount_spent_y'],axis=1)
orders = orders.rename({'amount_spent_x':'amount_spent'},axis=1)
orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,customer_label
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,Preferred
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,Preferred
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,Preferred


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [9]:
# 1. extract needed columns and create a new dataframe
# 2. identify unique customers
# 3. select only VIP customers
# 4. count the customers per country

vip_country = orders[['CustomerID','Country','customer_label']]
vip_country = vip_country.drop_duplicates(['CustomerID']).reset_index()
vip_country = vip_country[vip_country['customer_label']=='VIP']
vip_country['Country'].value_counts()

# the UK has the most VIP customers

United Kingdom     177
Germany             10
France               9
Switzerland          3
Portugal             2
Japan                2
Spain                2
EIRE                 2
Australia            2
Norway               1
Sweden               1
Netherlands          1
Cyprus               1
Singapore            1
Finland              1
Channel Islands      1
Denmark              1
Name: Country, dtype: int64

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [10]:
# 1. extract needed columns and create a new dataframe
# 2. identify unique customers
# 3. select only VIP and preferred customers
# 4. count the customers per country

vip_preferred_country = orders[['CustomerID','Country','customer_label']]
vip_preferred_country = vip_preferred_country.drop_duplicates(['CustomerID']).reset_index()
vip_preferred_country = vip_preferred_country[(vip_preferred_country['customer_label']=='VIP') | (vip_preferred_country['customer_label']=='Preferred')]
vip_preferred_country['Country'].value_counts()

# the UK has the most VIP and Preferred customers combined

United Kingdom     932
Germany             39
France              29
Belgium             11
Switzerland          9
Norway               7
Spain                7
Portugal             7
Italy                5
Finland              5
Australia            4
Channel Islands      4
Japan                4
Cyprus               3
Denmark              3
EIRE                 3
Sweden               2
Israel               2
Iceland              1
Netherlands          1
Poland               1
Greece               1
Singapore            1
Lebanon              1
Austria              1
Malta                1
Canada               1
Name: Country, dtype: int64